In [1]:
# %%capture
# #!unzip Datasets.zip

In [2]:
# from google.colab import drive

# # Mount the Google Drive
# drive.mount('/content/drive')

In [3]:
# %%capture
# !pip install datasets
# !pip install transformers
# !pip install librosa
# !pip install jiwer
# !pip install evaluate

In [4]:
import os
import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Set paths
csv_path = "/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/new+old.csv"  # Path to the CSV file
audio_folder = "/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves"  # Path to the folder containing .wav files
# Load the CSV
df = pd.read_csv(csv_path)
df = pd.read_csv(csv_path)
# Ensure the column names match
df.columns = ["Filename", "Transcription"]  # Rename columns if needed

# Append '.wav' to the file names
df['Filename'] = df['Filename'].apply(lambda x: f"{x}.wav")

# Add full paths to the audio files
df['file_path'] = df['Filename'].apply(lambda x: os.path.join(audio_folder, x))

# Verify that all audio files exist
missing_files = df[~df['file_path'].apply(os.path.exists)]
if not missing_files.empty:
    print("The following audio files are missing:")
    print(missing_files)
    raise FileNotFoundError("Some audio files listed in the CSV are missing in the folder.")

# Split into train (27) and test (3)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save splits to CSV for reference
train_csv_path = "train_split.csv"
test_csv_path = "test_split.csv"
train_df.to_csv(train_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Save HuggingFace datasets
train_dataset_path = "train_dataset"
test_dataset_path = "test_dataset"
train_dataset.save_to_disk(train_dataset_path)
test_dataset.save_to_disk(test_dataset_path)

# Output
print(f"Train set saved to: {train_csv_path} and {train_dataset_path}")
print(f"Test set saved to: {test_csv_path} and {test_dataset_path}")

/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Saving the dataset (1/1 shards): 100%|██████████| 1183/1183 [00:00<00:00, 594306.10 examples/s]

Train set saved to: train_split.csv and train_dataset
Test set saved to: test_split.csv and test_dataset


In [5]:
from datasets import load_from_disk

train_dataset = load_from_disk("train_dataset")
test_dataset = load_from_disk("test_dataset")

print(train_dataset)
print(test_dataset)

Dataset({
    features: ['Filename', 'Transcription', 'file_path', '__index_level_0__'],
    num_rows: 4730
})
Dataset({
    features: ['Filename', 'Transcription', 'file_path', '__index_level_0__'],
    num_rows: 1183
})


In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(train_dataset)

,Filename,Transcription,file_path,__index_level_0__
0,ishrat1-06_3.wav,مونم زِ دتُکھ بونہٕ پنس نار اماپوٗز,/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/ishrat1-06_3.wav,4049
1,8140313.wav,ابہامہٕ سٕتۍ پیدہ سبدِتھ ہیچمژ زِ شعرس,/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/8140313.wav,2975
2,khushboo_zehra123.wav,واقاتن پٮ۪ٹھ مبنی یِم زٕ مثنوۍ یہ چھے سانہ زبٲنۍ منز تھدٮ۪ن مرتبن ہٕنزِ حقدار,/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/khushboo_zehra123.wav,122
3,khushboo_zehra317.wav,پروفیسر عبدالعزیز سُند گُف چھُ زِ تہُند زاسنہٕ,/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/khushboo_zehra317.wav,316
4,mudasir1327.wav,مےٚ ہا وۅنۍ پکنہٕ ستۍ شانش ہِش زَن کھسان,/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/mudasir1327.wav,1826
5,Abid148.wav,بوز اصلی اوس کیا راز دروٗن,/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/Abid148.wav,647
6,8150260.wav,کاملنی ونۍ متی غزل تہٕ اکھ اکس نِشہ,/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/8150260.wav,3319
7,8140376.wav,کُن نظر مگر اتہ یییٚلہ نہٕ کانہہ تہٕ,/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/8140376.wav,3038
8,khushboo_zehra11.wav,اماپوز یہ سپدِتھ تہ ما چھے کانہہ طویل تہٕ دٔر روایتھ گال کھاتس تراوٕنۍ,/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/khushboo_zehra11.wav,10
9,Owais_II_103.wav,ٹیچر چھ مسلن حل کرنس منز مدد کران,/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/Owais_II_103.wav,2582


If there are any unwanted special characters in the dataset, we can remove them here, since there are none, I am keeping that as it is.

In [7]:
def extract_all_chars(batch):
  all_text = " ".join(batch["Transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)

Map: 100%|██████████| 1183/1183 [00:00<00:00, 200268.87 examples/s]


In [8]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'i': 1,
 '\xa0': 2,
 'آ': 3,
 'ا': 4,
 'ب': 5,
 'ت': 6,
 'ث': 7,
 'ج': 8,
 'ح': 9,
 'خ': 10,
 'د': 11,
 'ذ': 12,
 'ر': 13,
 'ز': 14,
 'س': 15,
 'ش': 16,
 'ص': 17,
 'ض': 18,
 'ط': 19,
 'ظ': 20,
 'ع': 21,
 'غ': 22,
 'ف': 23,
 'ق': 24,
 'ل': 25,
 'م': 26,
 'ن': 27,
 'و': 28,
 'َ': 29,
 'ُ': 30,
 'ِ': 31,
 'ْ': 32,
 'ٓ': 33,
 'ٔ': 34,
 'ٕ': 35,
 'ٖ': 36,
 'ٗ': 37,
 'ٚ': 38,
 'ٛ': 39,
 'ٮ': 40,
 'ٰ': 41,
 'ٲ': 42,
 'ٹ': 43,
 'پ': 44,
 'چ': 45,
 'ڈ': 46,
 'ڑ': 47,
 'ژ': 48,
 'ک': 49,
 'گ': 50,
 'ں': 51,
 'ھ': 52,
 'ہ': 53,
 'ۅ': 54,
 'ی': 55,
 'ۍ': 56,
 'ے': 57,
 '۪': 58}

In [9]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [10]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

61

In [11]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [12]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|", clean_up_tokenization_spaces=False)

In [13]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [14]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [15]:
train_dataset[0]["file_path"]

'/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/ishrat1-24_9.wav'

Replacing the File Path with Actual Audio.

In [16]:
from datasets import load_from_disk, Audio

# Load datasets
train_dataset = load_from_disk("train_dataset")  # Adjust to your actual path
test_dataset = load_from_disk("test_dataset")

# Rename 'file_path' to 'audio'
train_dataset = train_dataset.rename_column("file_path", "audio")
test_dataset = test_dataset.rename_column("file_path", "audio")

# # Cast the 'audio' column to use the Audio feature
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16_000))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16_000))

# # Drop unnecessary columns if needed
train_dataset = train_dataset.remove_columns(["__index_level_0__"])
test_dataset = test_dataset.remove_columns(["__index_level_0__"])

# # Verify the dataset structure
print(train_dataset)
print(test_dataset)

# # Inspect the first example
print(train_dataset[0])

Dataset({
    features: ['Filename', 'Transcription', 'audio'],
    num_rows: 4730
})
Dataset({
    features: ['Filename', 'Transcription', 'audio'],
    num_rows: 1183
})
{'Filename': 'ishrat1-24_9.wav', 'Transcription': 'ہاوو کیاہ سونتس کتھ أسۍ ویوٗر تُلو', 'audio': {'path': '/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/final-waves/ishrat1-24_9.wav', 'array': array([-0.17843628, -0.18099976, -0.19290161, ..., -0.07296753,
       -0.08895874, -0.07046509]), 'sampling_rate': 16000}}


In [15]:
#print(test_dataset[0]['audio'])

In [17]:
rand_int = random.randint(0, len(train_dataset))

print("Target text:", train_dataset[rand_int]["Transcription"])
print("Input array shape:", train_dataset[rand_int]["audio"]["array"].shape)
print("Sampling rate:", train_dataset[rand_int]["audio"]["sampling_rate"])

Target text: نہج برونٛہہ تہِ چھِ کٲشرِ زبٲنۍ منز کینژو لوٗکو رسالہ جاری کٔرۍمٕتۍ 
Input array shape: (129150,)
Sampling rate: 16000


In [18]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    batch["labels"] = processor(text=batch["Transcription"]).input_ids

    return batch

In [19]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)

In [20]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [21]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [22]:
import evaluate

wer_metric = evaluate.load("wer")

2025-09-07 10:50:25.687184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757222425.734004    2413 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757222425.751865    2413 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-07 10:50:25.864520: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using the latest cached version of the module from /home/muzaffar/.cache/huggingface/modules/evaluate_modules/metrics/evaluat

In [23]:
from evaluate import load

cer_metric = load("cer")


Using the latest cached version of the module from /home/muzaffar/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--cer/9cb90b752d5f15fb41161efdbefd13570adb3f32fa157290d8a55093c47428e1 (last modified on Mon Jan 20 00:59:28 2025) since it couldn't be found locally at evaluate-metric--cer, or remotely on the Hugging Face Hub.


**includes both WER and CER**

In [24]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    # Replace padding token (-100) with pad_token_id
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode predictions and labels to strings
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    if isinstance(label_str, list):
        if isinstance(pred_str, list) and len(pred_str) == len(label_str):
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str[index]}"')

        else:
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str}"')

    # Compute WER
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    # Compute CER
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}



In [25]:
#mmmmiiiiiinnnneeeee



# from transformers import Wav2Vec2ForCTC

# model = Wav2Vec2ForCTC.from_pretrained(
#     # "facebook/wav2vec2-xls-r-300m",
#     'facebook/wav2vec2-large-xlsr-53',
#     attention_dropout=0.05,
#     hidden_dropout=0.1,
#     feat_proj_dropout=0.1,
#     mask_time_prob=0.05,
#     layerdrop=0.01377,
#     gradient_checkpointing=True,
#     ctc_loss_reduction="mean",
#     ctc_zero_infinity=True,
#     pad_token_id=processor.tokenizer.pad_token_id,
#     vocab_size=len(processor.tokenizer),

    
# )

In [26]:

#pppeeeerrrrrseeaain-------iran

from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
   # 'facebook/wav2vec2-large-xlsr-53',
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.05,
    activation_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.01249,
    final_dropout=0.0,
    mask_time_prob=0.05,
    mask_time_length=10,
    mask_feature_prob=0,
    mask_feature_length=10,
    layerdrop=0.01377,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    ctc_zero_infinity=True,
    bos_token_id=processor.tokenizer.bos_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer.get_vocab())
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
model.freeze_feature_encoder()

In [26]:
# import huggingface_hub

# huggingface_hub.login()

In [27]:
#repo_name = "wav2vec2-kashmiri-jhon-data-one"

In [28]:
save_dir = "/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/training_all_details-300m"

In [29]:
#MMMMMMMIIIINNNEEEEE



from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=save_dir,
    group_by_length=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=10,
    learning_rate=4e-4,
    warmup_steps=250,
    save_total_limit=2,
    dataloader_num_workers=24
)

/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
# #PERSIAN

# from transformers import TrainingArguments

# training_args = TrainingArguments(
#   output_dir=repo_name,
#   group_by_length=True,
#   per_device_train_batch_size=2,
#   gradient_accumulation_steps=2,
#   eval_strategy="steps",
#   num_train_epochs=20,
#   gradient_checkpointing=True,
#   fp16=True,
#   save_steps=20,
#   eval_steps=20,
#   logging_steps=40,
#   learning_rate=3e-4,
#   warmup_steps=50,
#   save_total_limit=2,
#   push_to_hub=True,
# )

In [34]:
# import numpy as np
# from transformers import Trainer
# trainer = Trainer(
#     model=model,
#     data_collator=data_collator,
#     args=training_args,
#     compute_metrics=compute_metrics,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=processor.feature_extractor,
# )

In [30]:
import numpy as np
from transformers import Trainer

# Assuming processor is an instance of Wav2Vec2Processor (or similar for your model)
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class=processor,  # Use the processor directly for feature extraction
)




In [31]:
print("step1")
train_result = trainer.train()
print("step2")

metrics = train_result.metrics
print("step3")
max_train_samples = len(train_dataset)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))
print("step4")
trainer.save_model()
print("model created!")
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

step1


/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarn

Step,Training Loss,Validation Loss,Wer,Cer
500,2.405900,2.184669,1.000000,0.700952
1000,0.964900,0.852833,0.738076,0.238017
1500,0.648900,0.715902,0.636192,0.191809
2000,0.581600,0.686088,0.589938,0.177717
2500,0.422700,0.711799,0.582274,0.172498
3000,0.277000,0.763321,0.555946,0.163260
3500,0.231000,0.821042,0.549094,0.161937
4000,0.235800,0.879612,0.550086,0.163555
4500,0.170100,0.939581,0.530160,0.155413
5000,0.134400,0.949891,0.520242,0.152577


/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can

reference: "چشمن نظر یس کُن وُچھتھ شیہِلان ٲسم"
predicted: "ھمننزرکنوچھھل"
reference: "بودھ چھہ اتھ نِروانہٕ پتہٕ پٲدہٕ سپدن واجینۍ حالتھ مانان"
predicted: "گوچھکھرونپتپدھدنوھنمنن"
reference: "محی الدین مسکین سٕنز زیبا نگار"
predicted: "مدمکنسنھزبنر"


...

reference: "پننہ کھۅنہ منز للہٕ وٕہتھ"
predicted: "پننہ کھۅنہٕ منز للہٕ وٲوِتھ"
reference: "منٛز بڈٕہ خوش امد تہٕ زارہ پارٕہ کران ؤژھِس لاگتھ پژھس یا چیف"
predicted: "منز بڈِہ خوشامت تہٕ زارہ پار کران وژھس لٲگِتھ پٔزس یا چھے۪ٚ"
reference: "اوٚت وٲتِتھ ہیٚچھنوونس بہ تٔمہ واریاہ کتھہٕ"
predicted: "اوٚت وٲتِتھ ہش نونس بہٕ تٔمہِ واراہ کتھہٕ"


/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarn

reference: "مےٚ کوٚر سکول خٲطرٕ لنچ پیک"
predicted: "مےٚ کوٚر سکول خاطرٕ لنچ پیک"
reference: "رٲہینۍ شعر سۅخن یار چھِ بوٚمبراواں بوز"
predicted: "راینی شعر سوخن یار چھ بومبراوان بوز"
reference: "دۅہہٕ اکہ ژندٕ ہار گریسۍ گرِ"
predicted: "دۅہ اکہ ژندٕ ہار گریس گریز"


/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarn

step2
step3
step4
model created!
***** train metrics *****
  epoch                    =          30.0
  total_flos               = 23350245376GF
  train_loss               =          0.56
  train_runtime            =   15:03:08.97
  train_samples            =          4730
  train_samples_per_second =         2.619
  train_steps_per_second   =         0.164


In [32]:
trainer.evaluate()  # Evaluate the model on the test dataset

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/home/muzaffar/anaconda3/envs/tf14/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio input

reference: "زیادہٕ پوشہ ونۍ ڈاکٹر عروج زیدی وۅنۍ تمۍ سندین فارسی شعرن پٮ۪ٹھ چھہ پھرۍ پھرۍ حٲفز"
predicted: "زیادٕ پشونۍ ڈاکٹر اروٗر ذیدی وُنہٕ تمۍ سندین فارسی شٲعرن پٮ۪ٹھ چھہ پھرۍ پھرۍ حٲفی"
reference: "وزنس پٮ۪ٹھ چھہ نندرِ اندر نندر خراب تہٕ کٔر"
predicted: "وزنس پٮ۪ٹھ چھہ نیندرِ اندر نندر خراب تہٕ کٔر"
reference: "کٲشُر انساییکلو پیدیا کِس دویمِس جلدس منٛز چُھ أمِس پادشاہ سُند ناو عشوش سین"
predicted: "کٲشُر انساکلو پیدیا کِس دویمِس جلدس منٛز چُھ أمِس پادشاہ سُند ناو عشوش سین"


{'eval_loss': 1.1232091188430786,
 'eval_wer': 0.48940582454242176,
 'eval_cer': 0.14233024234937977,
 'eval_runtime': 114.019,
 'eval_samples_per_second': 10.375,
 'eval_steps_per_second': 1.298,
 'epoch': 30.0}

In [33]:
# trainer.push_to_hub()

# TESTING WITH OUT KENLM

In [34]:
import torch
import torchaudio
import librosa
import numpy
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import Wav2Vec2Processor



In [35]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["file_path"])
    speech_array = speech_array.squeeze().numpy()  # Convert to numpy array

    # Ensure the audio is always a 1D NumPy array (sometimes it's multi-channel)
    if len(speech_array.shape) > 1:  
        speech_array = np.mean(speech_array, axis=0)  # Convert stereo to mono
    
    # Resample to match the processor's expected sample rate
    speech_array = librosa.resample(
        y=np.asarray(speech_array), 
        orig_sr=sampling_rate, 
        target_sr=processor.feature_extractor.sampling_rate
    )

    batch["speech"] = speech_array.tolist()  # Convert to Python list (ensures consistency)
    return batch


In [36]:
model_name_or_path = "/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/training_all_details-300m/checkpoint-8500"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(model_name_or_path, device)

processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForCTC.from_pretrained(model_name_or_path).to(device)


def predict(batch):
    features = processor(
        batch["speech"],
        sampling_rate=processor.feature_extractor.sampling_rate,
        return_tensors="pt",
        padding=True
    )

    input_values = features.input_values.to(device)
    #attention_mask = features.attention_mask.to(device)
    attention_mask = features.attention_mask.to(device) if "attention_mask" in features else None


    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    pred_ids = torch.argmax(logits, dim=-1)

    batch["predicted_N_LM"] = processor.batch_decode(pred_ids)
    return batch





import torchaudio
import librosa
from datasets import load_dataset
import numpy as np

dataset = load_dataset("csv", data_files={"/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/test_split.csv"}, delimiter=",")

dataset = dataset.map(speech_file_to_array_fn)

/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment6/training_all_details-300m/checkpoint-8500 cuda


Generating train split: 1183 examples [00:00, 139370.31 examples/s]
Map: 100%|██████████| 1183/1183 [00:23<00:00, 50.99 examples/s]


In [37]:
result = dataset.map(predict, batched=True, batch_size=4)

Parameter 'function'=<function predict at 0x7fefd9936520> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 1183/1183 [01:57<00:00, 10.03 examples/s]


In [38]:
from evaluate import load  # Use `evaluate` instead of `datasets`

# Load WER and CER metrics
wer = load("wer")
cer = load("cer")

# Compute WER and CER using the correct split
print("WER: {:.2f}".format(100 * wer.compute(predictions=result["train"]["predicted_N_LM"],
                                             references=result["train"]["Transcription"])))
print("CER: {:.2f}".format(100 * cer.compute(predictions=result["train"]["predicted_N_LM"],
                                             references=result["train"]["Transcription"])))


Using the latest cached version of the module from /home/muzaffar/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--wer/85bee9e4216a78bb09b2d0d500f6af5c23da58f9210e661add540f5df6630fcd (last modified on Mon Jan 20 00:59:23 2025) since it couldn't be found locally at evaluate-metric--wer, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/muzaffar/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--cer/9cb90b752d5f15fb41161efdbefd13570adb3f32fa157290d8a55093c47428e1 (last modified on Mon Jan 20 00:59:28 2025) since it couldn't be found locally at evaluate-metric--cer, or remotely on the Hugging Face Hub.


WER: 49.27
CER: 14.40


In [39]:
for i in range(len(result["train"])):  # Specify the "train" split
    reference = result["train"]["Transcription"][i]  # Use "Text" as reference
    predicted_N_LM = result["train"]["predicted_N_LM"][i]

    if reference.strip() == predicted_N_LM.strip():
        continue

    print("Reference:", reference)
    print("Predicted:", predicted_N_LM)
    print('---')


Reference: کافر ان است اعتبار امروز اہلِ اِسلام ماندہ زار امروز
Predicted: کافر اِن استٕ اعتبار امروٗز اہلِ اِسلام ماع دزار امروٗز
---
Reference: کھٮ۪نہٕ خٲطرٕ کینژھا
Predicted: کھٮینہٕ خٲطرٕ کیژھاہ
---
Reference: ہٕندِ خٲطرٕ یِژھٕ کو ششہ ژٔراہمیت تھاوان بقول محمد امین کٲمل 
Predicted: ہٕندۍ خٲطر یِژ کو شیش ژرمیرتھاوانبقولِ محمود امین کٲمل
---
Reference: أسۍ للناواں لوٗکھ ہنٛز
Predicted: أسلل ناوان لوٗکھ نز
---
Reference: بنان اتھ وزنس منٛز چھہ زِ ماترا زِ
Predicted: بنان اتھ وزنس منٛز چھہ زماترا زِ
---
Reference: چھُ باقٕے قوتِ پرواز وُنہ میانٮ۪ن پرن اندر
Predicted: چھُ باقۍ کووتے پروازٕ وُنہ میانٮ۪ن پرن اندر
---
Reference: تہٕ فارسی یَس درمیان رشتہٕ روزِ ثابت پتہٕ ییٚلہِ غُل بمعنی شور 
Predicted: تہٕ فارسی یُس درمیان رشتہٕ روزِ صٲبت پتہٕ ییٚلہِ گُل بمعنۍ شور
---
Reference: ہوا ڈلہ وُن فضا خوش وُن بہارُک
Predicted: ہوا ڈل وُن فِضا خوشوُن بہارُک
---
Reference: یہِِ زانی یی نہٕ ادب تہٕ تہٕذیب کتھ
Predicted: یہِِ زٲنی یہِ نہٕ ادب تہٕ تہضیب کتھ
---
Reference: أمِس ٲسۍ زٕ نیچوۍ غلام محم

# CREATING KENLM

In [16]:
!git clone https://github.com/kpu/kenlm.git
%cd kenlm
!mkdir -p build
%cd build
!cmake .. && make
%cd ../..


Cloning into 'kenlm'...
remote: Enumerating objects: 14170, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 14170 (delta 85), reused 56 (delta 56), pack-reused 14047 (from 4)
Receiving objects: 100% (14170/14170), 5.98 MiB | 1.10 MiB/s, done.
Resolving deltas: 100% (8041/8041), done.
/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/kenlm
/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/kenlm/build
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr

In [17]:
import kenlm
import torch

In [18]:
!kenlm/build/bin/lmplz -o 5 \
    < "/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/final.txt" \
    > "/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/5-gram.arpa"




=== 1/5 Counting and sorting n-grams ===
Reading /home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/final.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 28283 types 9134
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:109608 2:2616782336 3:4906467328 4:7850347520 5:11448423424
Statistics:
1 9134 D1=0.732682 D2=1.09053 D3+=1.56773
2 24514 D1=0.881179 D2=1.32938 D3+=1.38179
3 26572 D1=0.957225 D2=1.57769 D3+=2.00899
4 24221 D1=0.979226 D2=1.81495 D3+=2.02077
5 21274 D1=0.958702 D2=1.87802 D3+=1.78901
Memory estimate for binary LM:
type      kB
probing 2370 assuming -p 1.5
probing 2847 assuming -r models -p 1.5
trie    1176 without quantization
trie     691 assuming -q 8 -b 8 quantization 
trie    1102 assuming -a 22 array pointer compression
trie  

In [19]:
import kenlm


In [43]:
#from pyctcdecode import build_ctcdecoder
from pyctcdecode import build_ctcdecoder
from transformers import AutoProcessor
from transformers import Wav2Vec2ProcessorWithLM

model_id = "/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/2training_all_details-xlsr53/checkpoint-4860"

# LM must be in arpa format (for now)
lm_path = "/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/5-gram.arpa"


processor = AutoProcessor.from_pretrained(model_id)
vocab_dict = processor.tokenizer.get_vocab()
sorted_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = build_ctcdecoder(
    list(sorted_dict.keys()),
    lm_path,
)

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

processor_with_lm.save_pretrained("/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/LLM  Testingv/language_model")

Loading the LM will be faster if you build a binary file.
Reading /home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/5-gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


# TESTING WITH KENLM

In [44]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import Wav2Vec2ProcessorWithLM
model_name_or_path = "/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/2training_all_details-xlsr53/checkpoint-4860"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(model_name_or_path, device)

processor = Wav2Vec2Processor.from_pretrained("/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/LLM  Testingv/")
model = Wav2Vec2ForCTC.from_pretrained(model_name_or_path).to(device)


def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["file_path"])
    speech_array = speech_array.squeeze().numpy()
    #speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)
    speech_array = librosa.resample(y=np.asarray(speech_array), orig_sr=sampling_rate, target_sr=processor.feature_extractor.sampling_rate)


    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = processor(
        batch["speech"],
        sampling_rate=processor.feature_extractor.sampling_rate,
        return_tensors="pt",
        padding=True
    )

    input_values = features.input_values.to(device)
    #attention_mask = features.attention_mask.to(device)
    attention_mask = features.attention_mask.to(device) if "attention_mask" in features else None


    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    pred_ids = torch.argmax(logits, dim=-1)

    batch["predicted_LM"] = processor.batch_decode(pred_ids)
    return batch

/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/experiment4/2training_all_details-xlsr53/checkpoint-4860 cuda


In [45]:
import torchaudio
import librosa
from datasets import load_dataset
import numpy as np

dataset = load_dataset("csv", data_files={"/home/muzaffar/Desktop/Research/papers/5-paper Wav2Vec/5. Wave2vec Whisper Paper/KASHMIRI/expriments3/test_split.csv"}, delimiter=",")

dataset = dataset.map(speech_file_to_array_fn)

In [46]:
result = dataset.map(predict, batched=True, batch_size=4)

Map: 100%|██████████| 650/650 [00:56<00:00, 11.48 examples/s]


In [47]:
from evaluate import load  # Use `evaluate` instead of `datasets`

# Load WER and CER metrics
wer = load("wer")
cer = load("cer")

# Compute WER and CER using the correct split
print("WER: {:.2f}".format(100 * wer.compute(predictions=result["train"]["predicted_LM"],
                                             references=result["train"]["Transcription"])))
print("CER: {:.2f}".format(100 * cer.compute(predictions=result["train"]["predicted_LM"],
                                             references=result["train"]["Transcription"])))

Using the latest cached version of the module from /home/muzaffar/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--wer/85bee9e4216a78bb09b2d0d500f6af5c23da58f9210e661add540f5df6630fcd (last modified on Mon Jan 20 00:59:23 2025) since it couldn't be found locally at evaluate-metric--wer, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/muzaffar/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--cer/9cb90b752d5f15fb41161efdbefd13570adb3f32fa157290d8a55093c47428e1 (last modified on Mon Jan 20 00:59:28 2025) since it couldn't be found locally at evaluate-metric--cer, or remotely on the Hugging Face Hub.


WER: 37.38
CER: 11.01


In [48]:
for i in range(len(result["train"])):  # Specify the "train" split
    reference = result["train"]["Transcription"][i]  # Use "Text" as reference
    predicted_LM = result["train"]["predicted_LM"][i]

    if reference.strip() == predicted_LM.strip():
        continue

    print("Reference:", reference)
    print("Predicted:", predicted_LM)
    print('---')

Reference: مصوتہ یتھ ڈپتھانگ ونان چھہِ پیش تہ زیر
Predicted: مصوتہٕ پٮ۪ٹھ ڈپتھانگ ونان چھہ پیش تہٕ زیر
---
Reference: مگر أکِس ہمکال توٲریخی واقعہٕہس کُن اشارٕ کر
Predicted: مگر أکس مہکال توٲریخی واقعہٕ ہس کُن اشارٕ
---
Reference: گۅڈنکِس جلدس منٛز چھُ لیکھنہٕ آمُت
Predicted: گۅڈنٕکِس جلدس منٛز چھہ لیکھنہٕ آمُت
---
Reference: آزاد س نِش واراہ استفادٕ کرُن پیومُت ستۍ ستۍ چھُ
Predicted: آزادس نِش واریاہ استفادہٕ کرُن پیومُت ستۍ ستۍ چُھ
---
Reference: تمٔۍ کرنٲو مثنوۍ تِتھے پٲٹھۍ
Predicted: تمۍ کرنٲو مثنوۍ تہٕ تھے پٲٹھۍ
---
Reference: ہرگاہ زَن یِتھٕ وٕے لُکو ہٕندۍ بارٕ تعداد
Predicted: ہرگاہ زن یِتھ وٕے لُکو ہٕندۍ بارٕ تعداد
---
Reference: مثنوۍ یَن ہُندیہ شاندار دور چھُ اَلہٕ پلہٕ ڈۅڈَس ہَتس ؤرۍ یَس پٮ۪ٹھ پھٔہل
Predicted: مثنوۍ ین ہُند یہ شاندار دور چُھ اَلہٕ پلہٕ ڈۅڈس ہَتس ؤرۍ یس پٮ۪ٹھ پھٔہل
---
Reference: شعر گوئی چھِ عام پٲٹھۍ بدیہہ گویی ونان
Predicted: شعر گویی چھہ عام پٲٹھۍ بدیعہ گویی ونان
---
Reference: ژھانڈن یلہ تمن ترٛاوِپتھ ونچ پوت کل
Predicted: واڈن یییٚلہ تمن ترٛاوِ پتھ 